## [분야1] 완전 자율 네트워크 운영을 위한 초단기 네트워크 트래픽 예측 베이스라인 코드
#### 주의: 반드시 본 파일을 이용하여 제출을 수행해야 하며 파일의 이름은 task.ipynb로 유지되어야 합니다.

### 추론 실행 환경
1. python 3.10 환경
2. torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu
    - 최신 버전입니다.

코드는 크게 3가지 파트로 구성되며, 해당 파트의 특성을 지켜서 내용을 편집하시면 되겠습니다.
1. 제출용 aifactory 라이브러리 및 추가 필요 라이브러리 설치
    - 채점 및 제출을 위한 aifactory 라이브러리를 설치하는 셀입니다. 이 부분은 수정하지 않고 그대로 실행합니다.
    - 필요 라이브러리를 직접 설치합니다.
2. 추론용 코드 작성
    - 모델 로드, 데이터 전처리, 예측 등 실제 추론을 수행하는 모든 코드를 이 영역에 작성합니다.
3. aif.submit() 함수를 호출하여 최종 결과를 제출하는 파트입니다.
    - 마이 페이지에서 발급받은 key 값을 함수의 인자로 정확히 입력해야 합니다.

※ 가능하면 제출시에는 포함되어 있는 train data를 폴더에서 제외하고 제출하시는 편이 좋습니다.
    - 파일 크기 감소 → 업로드 시간 감소 → 전체 추론 수행 시간 감소

### 1. 제출용 aifactory 라이브러리 설치
#### 결과 전송에 필요하므로 아래와 같이 aifactory 라이브러리가 반드시 최신버전으로 설치될 수 있게끔 합니다

In [1]:
!pip install -U aifactory

In [2]:
# 자신의 모델에 필요한 추가 라이브러리 설치
!pip install -U pytorch-lightning==2.0.9
!pip install -U scikit-learn==1.3.0
!pip install -U numpy==1.23.5
!pip install pexpect
!pip install decorator
!pip install certifi

### 2. 추론 환경의 기본 경로 구조
#### 제출 시 주의사항

1. 테스트 데이터 경로: /aif/data/test_inputs.pkl
   - 채점에 사용될 테스트 입력 데이터는 이 디렉토리 안에 test_inputs.pkl이라는 이름으로 고정되어 제공됩니다.
2. 모델 및 자원 경로: 예시 : ./model/
   - 추론 스크립트가 실행되는 위치를 기준으로, 제출된 모델 관련 파일들이 위치하는 상대 경로입니다.
   - 학습된 모델 가중치(.pt, .ckpt, .pth 등)
3. 제출 파일은 submission.pkl로 저장
4. argparse 사용시 args, _ = parser.parse_known_args()로 인자 지정
   args = parser.parse_args()는 jupyter에서 오류가 발생합니다!!!
5. return 할 결과물과 양식에 유의합니다.

In [23]:
import os
import numpy as np
import pandas as pd
import glob
import joblib
import torch
import torch.nn as nn
import pytorch_lightning as pl
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
import pickle
import json

# 0. 재현성을 위한 시드 고정
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# 디바이스 설정 (GPU 우선 사용)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 1. 테스트 데이터셋 클래스
class TrafficTestDataset(Dataset):
    def __init__(self, x_data, seq_len=10):
        self.data = x_data
        self.seq_len = seq_len
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return torch.FloatTensor(self.data[idx])

# 2. LSTM 모델 정의 
class LSTMModel(pl.LightningModule):
    def __init__(self, input_size, hidden_size=256, num_layers=1, dropout=0.2):  # train.py와 동일
        super().__init__()
        self.lstm = nn.LSTM(
            input_size, 
            hidden_size, 
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0,
            bidirectional=False  # train.py와 동일
        )
        self.dropout = nn.Dropout(dropout)
        # train.py와 정확히 동일한 FC 구조 (2층만!)
        self.fc1 = nn.Linear(hidden_size, hidden_size // 2)  # 256 → 128
        self.fc2 = nn.Linear(hidden_size // 2, 1)            # 128 → 1
        self.relu = nn.ReLU()
        
    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        last_output = lstm_out[:, -1, :]  # (batch, hidden_size)
        x = self.dropout(last_output)
        x = self.relu(self.fc1(x))        # 256 → 128
        x = self.dropout(x)               # dropout 추가
        x = self.fc2(x)                   # 128 → 1 (최종 출력)
        return x
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = nn.MSELoss()(y_hat, y)
        self.log('train_loss', loss)
        return loss
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = nn.MSELoss()(y_hat, y)
        self.log('val_loss', loss)
        return loss
    
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.001)

# 3. 저장된 모델과 스케일러 로드
def load_model_components(model_name="traffic_lstm_v7", 
                         model_path="./model"):
    # 메타데이터 로드
    with open(f'{model_path}/{model_name}_meta.json', 'r') as f:
        metadata = json.load(f)
    
    # X 스케일러 로드
    with open(f'{model_path}/{model_name}_scaler.pkl', 'rb') as f:
        x_scaler = pickle.load(f)
    
    # Y 스케일러 로드 (있다면)
    y_scaler_path = f'{model_path}/{model_name}_y_scaler.pkl'
    
    with open(y_scaler_path, 'rb') as f:
        y_scaler = pickle.load(f)
    
    
    # 모델 로드
    input_size = metadata.get('input_size', 27)
    hidden_size = metadata.get('hidden_size', 64)
    num_layers = metadata.get('num_layers', 1)
    dropout = metadata.get('dropout', 0.2)
    
    model = LSTMModel(input_size, hidden_size, num_layers, dropout)
    model.load_state_dict(torch.load(f'{model_path}/{model_name}.pth', map_location=device))
    model.to(device)
    model.eval()
    
    return model, x_scaler, y_scaler, metadata

# 4. 테스트 데이터 로드 및 전처리
try:
    # 실제 제출 환경
    test_data = joblib.load("/aif/data/test_inputs.pkl")
    print("실제 테스트 데이터 로드됨")
except FileNotFoundError:
    # 로컬 개발 환경
    print("로컬 환경: 샘플 데이터 시도")
    
    # 여러 방법으로 시도
    test_data = None
    methods = [
        ("joblib", lambda: joblib.load("./task1_data/test_inputs_sample.pkl")),
        ("pickle", lambda: pickle.load(open("./task1_data/test_inputs_sample.pkl", "rb"))),
        ("numpy", lambda: np.load("./task1_data/test_inputs_sample.pkl", allow_pickle=True))
    ]
    
    for method_name, loader in methods:
        try:
            test_data = loader()
            print(f"{method_name}로 샘플 데이터 로드 성공: {test_data.shape}")
            break
        except Exception as e:
            print(f"{method_name} 실패: {str(e)[:50]}...")
    
    # 모든 방법이 실패하면 더미 데이터 생성
    if test_data is None:
        print("더미 데이터 생성 중...")
        test_data = np.random.randn(100, 100, 27)  # (샘플수=100, 시퀀스=100, 피처=27)
        print(f"더미 데이터 생성 완료: {test_data.shape}")
# 모델 컴포넌트 로드
model_name = "traffic_lstm_v7"
model, x_scaler, y_scaler, metadata = load_model_components(model_name)

# 5. 테스트 데이터 전처리
processed_test_data = []

# test_data가 (163854, 100, 27) 형태의 numpy array인 경우
if isinstance(test_data, np.ndarray) and len(test_data.shape) == 3:
    # 효율적인 방법: 전체를 한번에 처리
    n_samples, original_seq_len, n_features = test_data.shape
    
    # 2D로 reshape하여 스케일링
    test_data_2d = test_data.reshape(-1, n_features)
    test_data_scaled_2d = x_scaler.transform(test_data_2d)
    test_data_scaled = test_data_scaled_2d.reshape(n_samples, original_seq_len, n_features)
    
    # 시퀀스 길이 조정 (SEQ_LEN에 맞춤)
    seq_len = metadata.get('seq_len', 10)
    
    for i in range(n_samples):
        sample = test_data_scaled[i]  # (100, 27)
        
        if seq_len <= original_seq_len:
            # 마지막 seq_len개만 사용
            sequence = sample[-seq_len:]
        else:
            # 패딩이 필요한 경우 (거의 없을 것)
            pad_len = seq_len - original_seq_len
            padding = np.zeros((pad_len, n_features))
            sequence = np.vstack([padding, sample])
            
        processed_test_data.append(sequence)
        
else:
    # test_data가 리스트 형태인 경우 (예전 방식)
    for sample in test_data:
        # 각 샘플을 개별적으로 처리
        sample_array = np.array(sample) if not isinstance(sample, np.ndarray) else sample
        
        # 스케일링 적용
        sample_scaled = x_scaler.transform(sample_array)
        
        # 시퀀스 길이 확인 및 조정
        seq_len = metadata.get('seq_len', 10)
        if len(sample_scaled) >= seq_len:
            sequence = sample_scaled[-seq_len:]
        else:
            pad_len = seq_len - len(sample_scaled)
            padding = np.zeros((pad_len, sample_scaled.shape[1]))
            sequence = np.vstack([padding, sample_scaled])
        
        processed_test_data.append(sequence)

# numpy array로 변환
processed_test_data = np.array(processed_test_data)

# 6. 데이터셋 및 데이터로더 생성 (DataLoader 대신 직접 배치 처리)
BATCH_SIZE = 256

# 7. 추론 실행 (DataLoader 없이)
y_pred = []

with torch.no_grad():
    # 직접 배치 처리
    for i in tqdm(range(0, len(processed_test_data), BATCH_SIZE), desc="[base_line]"):
        # 배치 데이터 추출
        batch_end = min(i + BATCH_SIZE, len(processed_test_data))
        batch_data = processed_test_data[i:batch_end]
        
        # 텐서로 변환
        x_batch = torch.FloatTensor(batch_data).to(device)
        
        # 모델 추론
        outputs = model(x_batch)  # (batch_size, 1)
        
        # CPU로 이동 및 numpy 변환
        outputs_cpu = outputs.cpu().numpy().flatten()  # (batch_size,)
        
        # y_scaler가 있으면 역변환
        if y_scaler is not None:
            outputs_original = y_scaler.inverse_transform(outputs_cpu.reshape(-1, 1)).flatten()
        else:
            outputs_original = outputs_cpu
        
        y_pred.append(outputs_original)

# 8. 결과 합치기
y_pred = np.concatenate(y_pred, axis=0)

# 결과가 2D인 경우 1D로 변환
if len(y_pred.shape) > 1:
    y_pred = y_pred.flatten()

print(f"예측 결과 shape: {y_pred.shape}")
print(f"예측값 샘플 (처음 5개): {y_pred[:5]}")

# 9. 결과 저장 (numpy array 형태)
joblib.dump(y_pred, "submission.pkl")
print("submission.pkl 파일로 저장 완료!")

로컬 환경: 샘플 데이터 시도
joblib로 샘플 데이터 로드 성공: (10, 100, 27)


[base_line]: 100%|██████████| 1/1 [00:00<00:00,  9.24it/s]

예측 결과 shape: (10,)
예측값 샘플 (처음 5개): [25617832. 25927488. 25950600. 25826522. 25806172.]
submission.pkl 파일로 저장 완료!


### 3. 제출하기 
#### ※ task별, 참가자별로 key가 다릅니다. 잘못 입력하지 않도록 유의바랍니다.
- key는 대회 페이지 [베이스라인 코드](https://aifactory.space/task/9162/baseline) 탭에 기재된 가이드라인을 따라 task별로 확인하실 수 있습니다.
- key가 틀리면 제출이 진행되지 않거나 잘못 제출되므로 태스크에 맞는 자기 key를 사용해야 합니다.

In [21]:
# 학습 데이터에서 peak_volume 분포 확인
train_data = pd.read_csv("./task1_data/train_data.csv")

print("=== Peak Volume 분석 ===")
print(f"Shape: {train_data.shape}")
print(f"Peak volume 통계:")
print(train_data['peak_volume'].describe())

print(f"\nTraffic volume vs Peak volume 비교:")
print(f"Traffic volume 평균: {train_data['traffic_volume'].mean():.2f}")
print(f"Peak volume 평균: {train_data['peak_volume'].mean():.2f}")

print(f"\n상위 10개 peak_volume 값:")
print(train_data['peak_volume'].nlargest(10).values)

print(f"\n컬럼들:")
print(train_data.columns.tolist())

=== Peak Volume 분석 ===
Shape: (700000, 28)
Peak volume 통계:
count    7.000000e+05
mean     3.368893e+07
std      2.489776e+07
min      1.553921e+06
25%      7.118103e+06
50%      2.839082e+07
75%      6.025272e+07
max      7.236918e+07
Name: peak_volume, dtype: float64

Traffic volume vs Peak volume 비교:
Traffic volume 평균: 29558116.84
Peak volume 평균: 33688930.67

상위 10개 peak_volume 값:
[72369180. 72369180. 72369180. 72369180. 72369180. 72369180. 72369180.
 72369180. 72369180. 72369180.]

컬럼들:
['fwd_pkt_count', 'bwd_pkt_count', 'fwd_tcp_pkt_count', 'bwd_tcp_pkt_count', 'fwd_udp_pkt_count', 'bwd_udp_pkt_count', 'traffic_volume', 'fwd_tcp_flags_cwr_count', 'bwd_tcp_flags_cwr_count', 'fwd_tcp_flags_ecn_count', 'bwd_tcp_flags_ecn_count', 'fwd_tcp_flags_ack_count', 'bwd_tcp_flags_ack_count', 'fwd_tcp_flags_push_count', 'bwd_tcp_flags_push_count', 'fwd_tcp_flags_reset_count', 'bwd_tcp_flags_reset_count', 'fwd_tcp_flags_syn_count', 'bwd_tcp_flags_syn_count', 'fwd_tcp_flags_fin_count', 'bwd_tcp_fl

In [8]:
print(f"예측 결과 shape: {y_pred.shape}")
print(f"예측값 범위: {y_pred.min():.0f} ~ {y_pred.max():.0f}")
print(f"예측값 평균: {y_pred.mean():.0f}")

예측 결과 shape: (10,)
예측값 범위: 25104890 ~ 30961964
예측값 평균: 26708882


In [24]:
import aifactory.score as aif
import time
t = time.time()

#-----------------------------------------------------#
aif.submit(model_name="ldrugsnw_20250813",
           key="e7ad0a52-1531-4b56-99ea-5891e04f56ce"
           )
#-----------------------------------------------------#
print(time.time() - t)

file : task.py
python
제출 완료
37.090944051742554
